### Load the .env file

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

### Import libraries

In [ ]:
import os

### Create Openai client

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

### Create an assistant

In [ ]:
client.beta.assistants.list()

In [ ]:
files = client.files.list()
print(files.model_dump_json(indent=2))

In [ ]:
client.files.create(
  file=open("2306.07691.pdf", "rb"),
  purpose="assistants"
)

In [ ]:
knowledge_retriever = client.beta.assistants.create(
    instructions="You are a helpful assistant, Use your knowledge base to best respond to customer queries.",
    name="Knowledge Retriever",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=["file-mctFiDKd1jXr8pgSPxRg2zJb"]
)

In [ ]:
print(knowledge_retriever.model_dump_json(indent=2))

### Create a new thread for the conversation

In [ ]:
thread = client.beta.threads.create()

In [ ]:
print(thread.model_dump_json(indent=2))

### Add message to the thread

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="can you explain style tts two?",
    file_ids=["file-mctFiDKd1jXr8pgSPxRg2zJb"]
)

In [ ]:
print(message.model_dump_json(indent=2))

### Run the query

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=knowledge_retriever.id,
  instructions="Please address the user as Raj."
)

In [ ]:
print(run.model_dump_json(indent=2))

### Check run status

In [ ]:
retribed_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [ ]:
print(retribed_run.model_dump_json(indent=2))

### Show the response

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)

In [ ]:
user = ''
assistant = ''

for tm in reversed(thread_messages.data):
    content = tm.content[0].text.value
    if tm.role == 'user':
        user += content
    if tm.role == 'assistant':
        assistant += content

In [ ]:
print(user)

In [ ]:
print(assistant)

### Delete the assistant

In [ ]:
print(client.beta.assistants.list())

In [ ]:
client.beta.assistants.delete('asst_KTCHWA7Ml6uQib2JhJrD8mSX')